<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>ModelOps demo - Python In-database Scaling and GLM using Git</b>
</header>

![image](images/git_meth.png) 

## Introduction

This Notebook will show you how to work with ClearScape Analytics in-database functions with ModelOps. With in-database analytics you can solve your scalable challenges by using Vantage to train and score your models. Wheter you have a big volume of data or you want to avoid the data movement implementation to train models outside Vantage, you can use ModelOps to manage your Catalog of Models from multiple platforms including in-database algorithms.

To know more about in-database algorithms review teradata official documentation

## Steps in this Notebook

<li>1. Configure the Environment </li>
    <li>2. Connect to Vantage</li>
    <li>3. Define Training function </li>
    <li>4. Define Evaluate function </li>
    <li>5. Define Scoring function</li>
    <li>6. Define Model Metadata</li>
    <li>7. Commit and Push to Git to let ModelOps manage</li>
    <li>8. ModelOps full lifecycle till deployment</li>
    <li>9. ModelOps Monitoring</li>


## Step 1. Configure the Environment

Here, we import the required libraries, set environment variables and environment paths (if required).

### 1.1 Libraries installation

A restart of the Kernel is needed to confirm changes


In [ ]:
%pip install -q teradataml==17.20.0.3 aoa==7.0.1 pandas==1.1.5 scikit-learn==0.24.2 matplotlib==3.5.2

### 1.2 Libraries import

In [ ]:
from teradataml import (
    create_context, 
    remove_context,
    get_context,
    get_connection,
    DataFrame,
    configure
)
import os
import getpass
import logging
import sys


## Step 2. Connect to Vantage

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=07_ModelOps_GIT_PIMA_Python_Indb_GLM.ipynb;' UPDATE FOR SESSION; ''')

# configure byom/val installation
configure.val_install_location = "VAL"
configure.byom_install_location = "MLDB"

# set the path to the local project repository for this model demo
model_local_path = '~/modelops-demo-models/model_definitions/pima_indb_glm'
res = os.system(f'mkdir -p {model_local_path}/model_modules')

## Step 3. Define Training Function

The training function takes the following shape

```python
def train(context: ModelContext, **kwargs):
    aoa_create_context()
    
    # your training code using teradataml indDB function
    model = <InDB Function>(...)
    
    # save your model
    model.result.to_sql(f"model_${context.model_version}", if_exists="replace")  
    
    record_training_stats(...)
```

You can execute this from the CLI or directly within the notebook as shown.

In [ ]:
%%writefile $model_local_path/model_modules/training.py
from teradataml import (
    DataFrame,
    GLM,
    ScaleFit,
    ScaleTransform
)
from aoa import (
    record_training_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)
import numpy as np


def plot_feature_importance(fi, img_filename):
    import pandas as pd
    import matplotlib.pyplot as plt
    feat_importances = pd.Series(fi)
    feat_importances.nlargest(10).plot(kind='barh').set_title('Feature Importance')
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()


def train(context: ModelContext, **kwargs):
    aoa_create_context()

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]
    entity_key = context.dataset_info.entity_key

    # read training dataset from Teradata and convert to pandas
    train_df = DataFrame.from_query(context.dataset_info.sql)

    print ("Scaling using InDB Functions...")
    
    scaler = ScaleFit(
        data=train_df,
        target_columns = feature_names,
        scale_method = context.hyperparams["scale_method"],
        miss_value = context.hyperparams["miss_value"],
        global_scale = context.hyperparams["global_scale"].lower() in ['true', '1'],
        multiplier = context.hyperparams["multiplier"],
        intercept = context.hyperparams["intercept"]
    )

    scaled_train = ScaleTransform(
        data=train_df,
        object=scaler.output,
        accumulate = [target_name,entity_key]
    )
    
    scaler.output.to_sql(f"scaler_${context.model_version}", if_exists="replace")
    print("Saved scaler")
    
    print("Starting training...")

    model = GLM(
        input_columns = feature_names,
        response_column = target_name,
        data = scaled_train.result,
        family = context.hyperparams["family"],
        learning_rate = context.hyperparams["learning_rate"],
        momentum = context.hyperparams["momentum"],
        initial_eta = context.hyperparams["initial_eta"],
        local_sgd_iterations = context.hyperparams["local_sgd_iterations"],
        iter_max = context.hyperparams["iter_max"],
        batch_size = context.hyperparams["batch_size"],
        iter_num_no_change = context.hyperparams["iter_num_no_change"]
    )
    
    model.result.to_sql(f"model_${context.model_version}", if_exists="replace")    
    print("Saved trained model")

    # Calculate feature importance and generate plot
    model_pdf = model.result.to_pandas()[['predictor','estimate']]
    predictor_dict = {}
    
    for index, row in model_pdf.iterrows():
        if row['predictor'] in feature_names:
            value = row['estimate']
            predictor_dict[row['predictor']] = value
    
    feature_importance = dict(sorted(predictor_dict.items(), key=lambda x: x[1], reverse=True))
    keys, values = zip(*feature_importance.items())
    norm_values = (values-np.min(values))/(np.max(values)-np.min(values))
    feature_importance = {keys[i]: float(norm_values[i]*1000) for i in range(len(keys))}
    plot_feature_importance(feature_importance, f"{context.artifact_output_path}/feature_importance")

    record_training_stats(
        train_df,
        features=feature_names,
        targets=[target_name],
        categorical=[target_name],
        feature_importance=feature_importance,
        context=context
    )

In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the training dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes
FROM PIMA_PATIENT_FEATURES F 
JOIN PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 <> 0
"""

feature_metadata =  {
    "database": "demo_user",
    "table": "aoa_statistics_metadata"
}

hyperparams = {
    "scale_method":"STD",
    "miss_value":"KEEP",
    "global_scale":"False",
    "multiplier":"1",
    "intercept":"0",
    "family": "BINOMIAL", 
    "learning_rate": "OPTIMAL",
    "momentum": 0.80,
    "initial_eta": 0.05,
    "local_sgd_iterations": 10,
    "iter_max": 100,
    "batch_size": 50,
    "iter_num_no_change": 5
}

entity_key = "PatientId"
target_names = ["HasDiabetes"]
feature_names = ["NumTimesPrg", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc", "Age"]

from aoa import ModelContext, DatasetInfo

dataset_info = DatasetInfo(
    sql=sql,
    entity_key=entity_key,
    feature_names=feature_names,
    target_names=target_names,
    feature_metadata=feature_metadata
)

ctx = ModelContext(
    hyperparams=hyperparams,
    dataset_info=dataset_info,
    artifact_output_path="artifacts/",
    model_version="InDB_v1",
    model_table="aoa_model_indb_v1"
)

sys.path.append(os.path.expanduser(f"{model_local_path}/model_modules"))
import training
training.train(context=ctx)

In [ ]:
# Check the generated files
!ls -lh artifacts

## Step 4. Define Evaluation Function

The evaluation function takes the following shape

```python
def evaluate(context: ModelContext, **kwargs):
    aoa_create_context()

    # read your model from Vantage
    model = DataFrame(f"model_${context.model_version}")
    
    # your evaluation logic
    
    record_evaluation_stats(...)
```

You can execute this from the CLI or directly within the notebook as shown.

In [ ]:
%%writefile $model_local_path/model_modules/evaluation.py
from sklearn.metrics import confusion_matrix
from teradataml import (
    copy_to_sql,
    DataFrame,
    TDGLMPredict,
    ScaleTransform,
    ClassificationEvaluator,
    ConvertTo,
    ROC
)
from aoa import (
    record_evaluation_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)
import json
import os
import numpy as np


def plot_feature_importance(fi, img_filename):
    import pandas as pd
    import matplotlib.pyplot as plt
    feat_importances = pd.Series(fi)
    feat_importances.nlargest(10).plot(kind='barh').set_title('Feature Importance')
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()
    
    
def plot_confusion_matrix(cf, img_filename):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(7.5, 7.5))
    ax.matshow(cf, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(cf.shape[0]):
        for j in range(cf.shape[1]):
            ax.text(x=j, y=i,s=cf[i, j], va='center', ha='center', size='xx-large')
    ax.set_xlabel('Predicted labels');
    ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix');
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()

    
def plot_roc_curve(roc_out, img_filename):
    import matplotlib.pyplot as plt
    auc = roc_out.result.to_pandas().reset_index()['AUC'][0]
    roc_results = roc_out.output_data.to_pandas()
    plt.plot(roc_results['fpr'], roc_results['tpr'], color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % 0.27)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()

    
def evaluate(context: ModelContext, **kwargs):

    aoa_create_context()

    model = DataFrame(f"model_${context.model_version}")

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]
    entity_key = context.dataset_info.entity_key

    test_df = DataFrame.from_query(context.dataset_info.sql)

    # Scaling the test set
    print ("Loading scaler...")
    scaler = DataFrame(f"scaler_${context.model_version}")

    scaled_test = ScaleTransform(
        data=test_df,
        object=scaler,
        accumulate = [target_name,entity_key]
    )
    
    print("Scoring")
    predictions = TDGLMPredict(
        object=model,
        newdata=scaled_test.result,
        accumulate=target_name,
        id_column=entity_key,
        output_prob=True,
        output_responses=['0','1']
    )

    predicted_data = ConvertTo(
        data = predictions.result,
        target_columns = [target_name,'prediction'],
        target_datatype = ["INTEGER"]
    )

    ClassificationEvaluator_obj = ClassificationEvaluator(
        data=predicted_data.result,
        observation_column=target_name,
        prediction_column='prediction',
        num_labels=2
    )

    metrics_pd = ClassificationEvaluator_obj.output_data.to_pandas()

    evaluation = {
        'Accuracy': '{:.2f}'.format(metrics_pd.MetricValue[0]),
        'Micro-Precision': '{:.2f}'.format(metrics_pd.MetricValue[1]),
        'Micro-Recall': '{:.2f}'.format(metrics_pd.MetricValue[2]),
        'Micro-F1': '{:.2f}'.format(metrics_pd.MetricValue[3]),
        'Macro-Precision': '{:.2f}'.format(metrics_pd.MetricValue[4]),
        'Macro-Recall': '{:.2f}'.format(metrics_pd.MetricValue[5]),
        'Macro-F1': '{:.2f}'.format(metrics_pd.MetricValue[6]),
        'Weighted-Precision': '{:.2f}'.format(metrics_pd.MetricValue[7]),
        'Weighted-Recall': '{:.2f}'.format(metrics_pd.MetricValue[8]),
        'Weighted-F1': '{:.2f}'.format(metrics_pd.MetricValue[9]),
    }

    with open(f"{context.artifact_output_path}/metrics.json", "w+") as f:
        json.dump(evaluation, f)
        
    cm = confusion_matrix(predicted_data.result.to_pandas()['HasDiabetes'], predicted_data.result.to_pandas()['prediction'])
    plot_confusion_matrix(cm, f"{context.artifact_output_path}/confusion_matrix")

    roc_out = ROC(
        data=predictions.result,
        probability_column='prob_1',
        observation_column=target_name,
        positive_class='1',
        num_thresholds=1000
    )
    plot_roc_curve(roc_out, f"{context.artifact_output_path}/roc_curve")

    # Calculate feature importance and generate plot
    model_pdf = model.to_pandas()[['predictor','estimate']]
    predictor_dict = {}
    
    for index, row in model_pdf.iterrows():
        if row['predictor'] in feature_names:
            value = row['estimate']
            predictor_dict[row['predictor']] = value
    
    feature_importance = dict(sorted(predictor_dict.items(), key=lambda x: x[1], reverse=True))
    keys, values = zip(*feature_importance.items())
    norm_values = (values-np.min(values))/(np.max(values)-np.min(values))
    feature_importance = {keys[i]: float(norm_values[i]*1000) for i in range(len(keys))}
    plot_feature_importance(feature_importance, f"{context.artifact_output_path}/feature_importance")

    predictions_table = "predictions_tmp"
    copy_to_sql(df=predicted_data.result, table_name=predictions_table, index=False, if_exists="replace", temporary=True)

    # calculate stats if training stats exist
    if os.path.exists(f"{context.artifact_input_path}/data_stats.json"):
        record_evaluation_stats(
            features_df=test_df,
            predicted_df=DataFrame.from_query(f"SELECT * FROM {predictions_table}"),
            feature_importance=feature_importance,
            context=context
        )

In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the evaluation dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes 
FROM PIMA_PATIENT_FEATURES F 
JOIN PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 = 0
"""

entity_key = "PatientId"
target_names = ["HasDiabetes"]
feature_names = ["NumTimesPrg", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc", "Age"]
 
dataset_info = DatasetInfo(
    sql=sql,
    entity_key=entity_key,
    feature_names=feature_names,
    target_names=target_names,
    feature_metadata=feature_metadata
)

ctx = ModelContext(
    hyperparams=hyperparams,
    dataset_info=dataset_info,
    artifact_output_path="artifacts/",
    artifact_input_path="artifacts/",
    model_version="InDB_v1",
    model_table="aoa_model_indb_v1"
)

import evaluation
evaluation.evaluate(context=ctx)

# view evaluation results
import json
with open(f"{ctx.artifact_output_path}/metrics.json") as f:
    print(json.load(f))

In [ ]:
# Check the generated files
!ls -lh artifacts

## Step 5. Define Scoring Function

The scoring function takes the following shape

```python
def score(context: ModelContext, **kwargs):
    aoa_create_context()

    # read your model
    model = DataFrame(f"model_${context.model_version}")
    
    # your evaluation logic
    
    record_scoring_stats(...)
```

You can execute this from the CLI or directly within the notebook as shown.

In [ ]:
%%writefile $model_local_path/model_modules/scoring.py
from teradataml import (
    copy_to_sql,
    DataFrame,
    TDGLMPredict,
    ScaleTransform
)
from aoa import (
    record_scoring_stats,
    aoa_create_context,
    ModelContext
)
import pandas as pd


def score(context: ModelContext, **kwargs):

    aoa_create_context()

    model = DataFrame(f"model_${context.model_version}")

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]
    entity_key = context.dataset_info.entity_key

    features_tdf = DataFrame.from_query(context.dataset_info.sql)
    features_pdf = features_tdf.to_pandas(all_rows=True)

    # Scaling the scoring set
    print ("Loading scaler...")
    scaler = DataFrame(f"scaler_${context.model_version}")

    scaled_features = ScaleTransform(
        data=features_tdf,
        object=scaler,
        accumulate = entity_key
    )
    
    print("Scoring")
    predictions = TDGLMPredict(
        object=model,
        newdata=scaled_features.result,
        id_column=entity_key
    )

    predictions_pdf = predictions.result.to_pandas(all_rows=True).rename(columns={"prediction": target_name}).astype(int)

    print("Finished Scoring")

    # store the predictions
    predictions_pdf = pd.DataFrame(predictions_pdf, columns=[target_name])
    predictions_pdf[entity_key] = features_pdf.index.values
    # add job_id column so we know which execution this is from if appended to predictions table
    predictions_pdf["job_id"] = context.job_id

    # teradataml doesn't match column names on append.. and so to match / use same table schema as for byom predict
    # example (see README.md), we must add empty json_report column and change column order manually (v17.0.0.4)
    # CREATE MULTISET TABLE pima_patient_predictions
    # (
    #     job_id VARCHAR(255), -- comes from airflow on job execution
    #     PatientId BIGINT,    -- entity key as it is in the source data
    #     HasDiabetes BIGINT,   -- if model automatically extracts target
    #     json_report CLOB(1048544000) CHARACTER SET UNICODE  -- output of
    # )
    # PRIMARY INDEX ( job_id );
    predictions_pdf["json_report"] = ""
    predictions_pdf = predictions_pdf[["job_id", entity_key, target_name, "json_report"]]

    copy_to_sql(
        df=predictions_pdf,
        schema_name=context.dataset_info.predictions_database,
        table_name=context.dataset_info.predictions_table,
        index=False,
        if_exists="append"
    )
    
    print("Saved predictions in Teradata")

    # calculate stats
    predictions_df = DataFrame.from_query(f"""
        SELECT 
            * 
        FROM {context.dataset_info.get_predictions_metadata_fqtn()} 
            WHERE job_id = '{context.job_id}'
    """)

    record_scoring_stats(features_df=features_tdf, predicted_df=predictions_df, context=context)


In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the scoring dataset 

sql = """
SELECT 
    F.*
FROM PIMA_PATIENT_FEATURES F 
    WHERE F.patientid MOD 5 = 0
"""

# where to store predictions
predictions = {
    "database": "demo_user",
    "table": "pima_patient_predictions_tmp"
}

import uuid
job_id=str(uuid.uuid4())

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata,
                           predictions=predictions)

ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="artifacts/",
                   artifact_input_path="artifacts/",
                   model_version="InDB_v1",
                   model_table="aoa_model_indb_v1",
                   job_id=job_id)

import scoring
scoring.score(context=ctx)

In [ ]:
DataFrame.from_query(f"SELECT * FROM pima_patient_predictions_tmp WHERE job_id='{job_id}'")

In [ ]:
# Clean up

os.system('rm -f artifacts/*')

try:
    get_context().execute(f"DROP TABLE model_InDB_v1")
except: 
    pass

try:
    get_context().execute(f"DROP TABLE scaler_InDB_v1")
except: 
    pass

try:
    get_context().execute(f"DROP TABLE pima_patient_predictions_tmp")
except: 
    pass

## Step 6. Define Model Metadata

Now let's create the configuration files.

Requirements file with the dependencies and versions:

In [ ]:
%%writefile $model_local_path/model_modules/requirements.txt
teradataml==17.20.0.3
aoa==7.0.1
pandas==1.1.5
scikit-learn==0.24.2
matplotlib==3.5.2

The hyper parameter configuration (default values):

In [ ]:
%%writefile $model_local_path/config.json
{
   "hyperParameters": {
        "scale_method":"STD",
        "miss_value":"KEEP",
        "global_scale": "False",
        "multiplier":"1",
        "intercept":"0",
        "family": "BINOMIAL", 
        "learning_rate": "OPTIMAL",
        "momentum": 0.80,
        "initial_eta": 0.05,
        "local_sgd_iterations": 10,
        "iter_max": 100,
        "batch_size": 50,
        "iter_num_no_change": 5
   }
}

The model configuration:

In [ ]:
%%writefile $model_local_path/model.json
{
    "id": "f8df0bec-12d1-4d2d-920f-4448503df82d",
    "name": "Python PIMA InDB GLM",
    "description": "Python PIMA InDB GLM for Diabetes Prediction",
    "language": "python"
}

## Step 7.Commit and Push to Git to let ModelOps manage

Run the command below to commit and push changes to our forked repository, so ModelOps can fetch the changes to the model.

In [ ]:
!cd $model_local_path/../.. && git add . && git commit -m "Added Python PIMA InDB GLM demo model 🚀" && git push

Now that changes are pushed, you can make the lifecycle inside **ModelOps User Interface**, plan for new trainings, evaluations, scorings. Compare models and operationalize into Production with automated Monitoring and alerting capabilities.

## Step 8. ModelOps full lifecycle till deployment


Use or Create a Project with the git code repository with the model code, then you should see the model in the catalog already created

<img src="images/08_01.png" alt="Model Catalog with inDB"/>

Select the Model and then click Train a new Model. Use default hyper-parameters. This will launch the training job with the training script we generated and pushed to Git.

<img src="images/08_02.png" alt="Train"/>

<img src="images/08_03.png" alt="Train job" width="500" height="500"/>

<img src="images/08_04.png" alt="Train finished" width="500" height="500"/>

When Model is trained a new Model Id is created and you can get inside the Model Lifecycle screen to review artifacts and other details

<img src="images/08_06.png" alt="Model lifecycle"/>

Now, let's evaluate the Model, click the button and select the evaluation dataset. This will launch the evaluation job with the training script we generated and pushed to Git.

<img src="images/08_07.png" alt="Evaluation" width="500" height="500"/> <img src="images/08_08.png" alt="Evaluation job" width="500" height="500"/>



When evaluation job is finished a Model evaluation Report is generated with the metrics and charts that evaluation script generates

<img src="images/08_26.png" alt="Model Report" />

Now, let's approve the model and provide an approval description

<img src="images/08_09.png" alt="Approval" />

<img src="images/08_10.png" alt="Approval description" width="500" height="500"/>

The model is ready to be deployed. Let's deploy using a Batch scheduling option - Run it manual

<img src="images/08_11.png" alt="Deployment Engine" width="500" height="500"/>

<img src="images/08_12.png" alt="Deployment Publish" width="500" height="500"/>

<img src="images/08_13.png" alt="Deployment Schedule" width="500" height="500"/>


Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

## Step 9. ModelOps Monitoring

Now the model is deployed and a new Deployment appears in the deployment screen


<img src="images/08_15.png" alt="Deploymet" />


You can run jobs manually from here, review history of executions and view the predictions for a specific job

<img src="images/08_16.png" alt="Deployment Run" width="500" height="500"/>

<img src="images/08_17.png" alt="Deployment Jobs" />

<img src="images/08_18.png" alt="Deployment view" width="500" height="500" />

<img src="images/08_19.png" alt="Deployment predictions" width="500" height="500"/>

<img src="images/08_20.png" alt="Deployment" width="500" height="500"/>


From the Feature Drift and Prediction Drift tabs you can check on the monitoring of the data drift

<img src="images/08_22.png" alt="Feature Drift" />

<img src="images/08_21.png" alt="Prediction Drift" />

<img src="images/08_23.png" alt="Performance Monitoring" />




From the Performance Drift, you can review multiple evaluations, let's evaluate the model with a new dataset. We create a new evaluation dataset with this query:
    
    SELECT * FROM pima_patient_diagnoses F WHERE F.patientid MOD 8 <> 0
    
<img src="images/08_24.png" alt="Evaluate" width="500" height="500" />

and now see the evolution of the metrics

<img src="images/08_25.png" alt="Metrics monitoring" />


With ModelOps you can close the cycle and review make decisions when you need to replace yor model in production, For example, You could get alerting from Data Drift of Performance Drift and you can create multiple versions and compare them, select a champion and deploy new versions that replace existing in Production.

Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>